In [1]:
try:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.common.keys import Keys
    from webdriver_manager.chrome import ChromeDriverManager
    import csv
    import time
    import os,pyautogui
except:
    print("Any Module is not download")


url = 'https://www.linkedin.com/jobs/search/?currentJobId=3308450031&geoId=&keywords=Data%20Scientist&location=canada'
co = webdriver.ChromeOptions()
co.add_argument("--start-maximized")
co.add_argument('--disable-gpu')
co.add_argument('--disable-infobars')
co.add_experimental_option("detach", True)
    
     
driver = webdriver.Chrome(ChromeDriverManager().install(),options=co)
driver.get(url)
actions = ActionChains(driver)

C:\Users\ABC\AppData\Local\Temp\ipykernel_7844\3203150882.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=co)


In [ ]:
actions = ActionChains(driver)

file = open('linkedin_scraper.csv','w', newline='',encoding="utf-8")
writer = csv.writer(file)
writer.writerow(["Title", "Location", "Company", "Job description", "Employement Type"])

cnt = 0

while True:
    try:
        while True:
            
            scraper = driver.find_elements(By.XPATH,"//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")

            if cnt == 0:
                last_job = scraper[cnt]
            else:
                last_job = scraper[cnt-1]

            job = scraper[cnt]

            actions.move_to_element(job).perform()
            job.click()

            # time.sleep(1)

            try:
                location = WebDriverWait(driver,1).until(EC.visibility_of_element_located((By.XPATH,"//span[@class='topcard__flavor topcard__flavor--bullet']"))).text
            except:
                while True:
                    try:
                        actions.move_to_element(last_job).perform()
                        actions.click(last_job).perform()

                        # time.sleep(1)

                        actions.move_to_element(job).perform()
                        actions.click(job).perform()
                        location = WebDriverWait(driver,1.5).until(EC.visibility_of_element_located((By.XPATH,"//span[@class='topcard__flavor topcard__flavor--bullet']"))).text
                        break
                    except:pass

            title = driver.find_element(By.XPATH,"//h2").text
            company = driver.find_element(By.XPATH,"//span[@class='topcard__flavor']").text
            Employement_type = driver.find_element(By.XPATH,"//h3[contains(text(),'Employment type')]/../span").text

            try:
                #clicks on see more
                actions.move_to_element(WebDriverWait(driver,1).until(EC.presence_of_element_located((By.XPATH,"//button[@data-tracking-control-name='public_jobs_show-more-html-btn']")))).click().perform()
            except:pass

            details = driver.find_element(By.XPATH, "//*[@class='description__text description__text--rich']").text

            last_job = job

            writer.writerow([title, location, company, details, Employement_type])

            cnt += 1

    except:
            try:
                WebDriverWait(driver,0.5).until(EC.visibility_of_element_located((By.XPATH, "//*[@aria-label='Load more results']"))).click()
            except:
                try:
                    WebDriverWait(driver,0.5).until(EC.presence_of_element_located((By.XPATH,"//h3[@class='t-16 t-black t-bold pt4 pb2 ']")))
                except:pass
